In [1]:
import pandas as pd
import sys
import requests as r

sys.path.append('../helper_functions')
import github_utils as gh
import tokenlist_utils as tl
import duneapi_utils as dapi

url_content = [
    # owner_name, repo_name, path_name, file_name
      ['ethereum-optimism','ethereum-optimism.github.io','','optimism.tokenlist.json',] #OP Bridge - https://github.com/ethereum-optimism/ethereum-optimism.github.io/blob/master/optimism.tokenlist.json
    , ['rainbow-me','rainbow-token-list','output','lean-rainbow-token-list.json'] #Rainbow - https://github.com/rainbow-me/rainbow-token-list/blob/main/output/lean-rainbow-token-list.json
    , ['velodrome-finance','docs','public','tokenlist.json'] #Velo - https://github.com/velodrome-finance/docs/blob/aa557db47248b88af73dcdebfbc161e2b93da712/public/tokenlist.json#L4
]

In [2]:
dune_list_query_id = 2420477 #https://dune.com/queries/2420477

In [3]:
# Load the JSON data from the URL
df_list = []
for gh_url in url_content:

    owner_name = gh_url[0]
    repo_name = gh_url[1]
    path_name = gh_url[2]
    file_name = gh_url[3]

    gh_file = gh.get_file_url_from_github(owner_name,repo_name,path_name,file_name)
    res = r.get(gh_file)
    data = res.json()
    tmp = tl.generate_table_from_tokenlist(data)
    tmp['list_name'] = owner_name
    df_list.append(tmp)

df = pd.concat(df_list)
df = df[df['chainId'] ==10] #Get OP Version

df = df.groupby(['chainId', 'address', 'name', 'symbol', 'decimals'])['list_name'].agg(list)
df = df.reset_index()
# display(df)

In [4]:
dune_list = dapi.get_dune_data(dune_list_query_id)
# display(dune_list)

2023-09-17 09:01:59.141 | INFO     | duneapi_utils:get_dune_data:44 - Results available at https://dune.com/queries/2420477
2023-09-17 09:02:00.402 | INFO     | duneapi_utils:get_dune_data:89 - ✨ Results saved as csv_outputs/my_query_results.csv, with 560 rows and 4 columns.


In [5]:
#token addresses to lower
df['contract_address'] = df['address'].str.lower().str.strip()
dune_list['contract_address'] = dune_list['contract_address'].str.lower().str.strip()

In [6]:
adds = df[~df['contract_address'].isin(dune_list['contract_address'])]
# display(adds[['address','symbol','decimals', 'name','list_name']])


In [7]:

# Assuming your DataFrame is named 'adds'
for index, row in adds.iterrows():
    sql_values_str = f",({row['address']}, '{row['symbol']}', {row['decimals']}, 'underlying')"
    print(sql_values_str)

,(0x03e5b5930f3a42b56af4a982b10e60957bdd2f61, 'D2D', 18, 'underlying')
,(0x629c2fd5d5f432357465b59d7832389a89956f0b, 'COC', 18, 'underlying')
,(0x6c518f9D1a163379235816c543E62922a79863Fa, 'bwAJNA', 18, 'underlying')
,(0x76c37F9949e05b37C8373d155C1Fef46a6858481, 'ePendle', 18, 'underlying')
,(0x8368Dca5CE2a4Db530c0F6e535d90B6826428Dee, 'FPIS', 18, 'underlying')
,(0x8637725aDa78db0674a679CeA2A5e0A0869EF4A1, 'NFTE', 18, 'underlying')
,(0x96bBD55479e9923512dcc95Eb7Df5edDe6FB9874, 'wOpenX', 18, 'underlying')
,(0xAd42D013ac31486B73b6b059e748172994736426, '1INCH', 18, 'underlying')
,(0xC52D7F23a2e460248Db6eE192Cb23dD12bDDCbf6, 'crvUSD', 18, 'underlying')
,(0xa925f4057d6E6C8FAf8bdE537Ad14BA91A1D0337, 'SYNTH', 18, 'underlying')
,(0xaf3A6f67Af1624d3878A8d30b09FAe7915DcA2a0, 'EQB', 18, 'underlying')
,(0xc5d43a94e26fca47a9b21cf547ae4aa0268670e1, 'FPI', 18, 'underlying')
,(0xc871cCf95024eFa2CbcE69B5B775D2a1DcF49c1B, 'GROW', 18, 'underlying')
